In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import pandas as pd
import shutil
import matplotlib.image as mpimg
from tqdm import tqdm
from PIL import Image
import numpy as np

In [ ]:
import constants

labels = pd.read_csv(constants.LABELS_PATH_CROPPED)
labels.head()

In [ ]:
emotive = labels[labels['label']!= "Neutral"]

# I want to maintain the original label for analysis at the end of the model
emotive.loc[:, 'label'] = 'Emotive'
neutral = labels[labels['label'] == "Neutral"]
neutral.loc[:, 'label'] = 'Neutral'
print(f"{len(emotive) = }")
print(f"{len(neutral) = }")

In [ ]:
df_binary = pd.concat([emotive, neutral])
print(df_binary.head())
print(df_binary.tail())

In [ ]:
LABEL_ENCODE_DICT = {
    'Emotive': 1,
    'Neutral': 0
}

In [ ]:
def encode_label(df, encoder_dict):
    """returns df where label column is encoded"""
    df['label_encoded'] = df['label'].map(encoder_dict)
    return df

encode_label(df_binary, LABEL_ENCODE_DICT)

In [ ]:
import preprocessing

In [ ]:
X, y = preprocessing.preprocess_data_part1(df_binary, constants.IMAGES_FOLDER_PATH)
X_grey, y_grey = preprocessing.preprocess_data_part1(df_binary, constants.IMAGES_FOLDER_PATH, greyscale=True)

print(f"images shape {X.shape}")
print(f"y shape {y.shape}")
print(f"grey images shape {X_grey.shape}")
print(f"grey y shape {y_grey.shape}")

In [ ]:
# Choose 5 random indices
random_indices = np.random.choice(X.shape[0], size=5, replace=False)

# Create a figure and axes
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
# Iterate over the random indices and display the images
for i, idx in enumerate(random_indices):
    # Display the image
    axes[i].imshow(X[idx] / 255.0)  # Scale pixel values to [0, 1] for display
    axes[i].set_title(f"Label: {y[idx]}\nShape: {X[idx].shape[0]}x{X[idx].shape[1]}")
    axes[i].axis('off')

plt.show()

In [ ]:
# Create a figure and axes
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
# Iterate over the random indices and display the images
for i, idx in enumerate(random_indices):
    # Display the image
    axes[i].imshow(X_grey[idx] / 255.0)  # Scale pixel values to [0, 1] for display
    axes[i].set_title(f"Label: {y_grey[idx]}\nShape: {X_grey[idx].shape[0]}x{X_grey[idx].shape[1]}")
    axes[i].axis('off')

plt.show()

In [ ]:
# define splits
split = (0.6, 0.2, 0.2)
X_train, y_train, X_val, y_val, X_test, y_test = preprocessing.data_split_and_augment(X, y, split)

X_train_grey, y_train_grey, X_val_grey, y_val_grey, X_test_grey, y_test_grey = preprocessing.data_split_and_augment(X_grey, y_grey, split)


print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print(f"X_val shape {X_val.shape}")
print(f"y_val shape {y_val.shape}")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")
print(f"X_train_grey shape {X_train_grey.shape}")
print(f"X_val_grey shape {X_val_grey.shape}")
print(f"X_test_grey shape {X_test_grey.shape}")

In [ ]:
# delete the X and Y arrays to free up more space in RAM
del X
del y

In [ ]:
n, bins, patches = plt.hist(y_train, bins=2, edgecolor='black', align='mid')

# Set colors for the bars
colors = ['skyblue', 'salmon']
for i, patch in enumerate(patches):
    patch.set_facecolor(colors[i])

# Set labels for the x-axis and y-axis
plt.xlabel('Class')
plt.ylabel('Frequency')

# Set ticks to show labels for classes
plt.xticks([.25, .75], ['Neutral', 'Emotive'])

# Set the title of the plot
plt.title('Distribution of Neutral vs. Emotive')

# Add labels in the middle of the bars
for patch in patches:
    height = patch.get_height()
    # Calculate the x position of the label (middle of the bar)
    x = patch.get_x() + patch.get_width() / 2
    # Add the label with the height value
    plt.text(x, height, str(int(height)), ha='center', va='bottom')

# Show the plot
plt.show()

In [ ]:
# Count the number of 'Neutral' instances
neutral_count = np.sum(y_train == 0)

# Find indices of 'Emotive' instances
emotive_indices = np.where(y_train == 1)[0]

# Randomly select 'Emotive' indices to remove to match the 'Neutral' count
indices_to_remove = np.random.choice(emotive_indices, size=len(emotive_indices) - neutral_count, replace=False)

# Remove the selected 'Emotive' instances from X_train and y_train
X_train_balanced = np.delete(X_train, indices_to_remove, axis=0)
y_train_balanced = np.delete(y_train, indices_to_remove, axis=0)

X_train_balanced_grey = np.delete(X_train_grey, indices_to_remove, axis=0)
y_train_balanced_grey = np.delete(y_train_grey, indices_to_remove, axis=0)


n, bins, patches = plt.hist(y_train_balanced, bins=2, edgecolor='black', align='mid')

# Set colors for the bars
colors = ['skyblue', 'salmon']
for i, patch in enumerate(patches):
    patch.set_facecolor(colors[i])

# Set labels for the x-axis and y-axis
plt.xlabel('Class')
plt.ylabel('Frequency')

# Set ticks to show labels for classes
plt.xticks([.25, .75], ['Neutral', 'Emotive'])

# Set the title of the plot
plt.title('Distribution of Neutral vs. Emotive')

# Add labels in the middle of the bars
for patch in patches:
    height = patch.get_height()
    # Calculate the x position of the label (middle of the bar)
    x = patch.get_x() + patch.get_width() / 2
    # Add the label with the height value
    plt.text(x, height, str(int(height)), ha='center', va='bottom')

# Show the plot
plt.show()

print(X_train_balanced.shape)
print(y_train_balanced.shape)

### Baseline Model

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

class MajorityClassModel:
    """ A dummy model that always predicts the majority class """
    def __init__(self, majority_class):
        self.majority_class = majority_class

    def predict(self, X):
        return np.array([self.majority_class] * len(X))

    def evaluate(self, X, y_true):
        y_pred = self.predict(X)
        accuracy = round(accuracy_score(y_true, y_pred), 4)
        return accuracy

In [ ]:
# Find the most frequent value in y_train (
class_frequencies = np.unique(y_train, return_counts=True)
majority_class = class_frequencies[0][np.argmax(class_frequencies[1])]
print(f"{majority_class = }")

# Initialize the model
base_model = MajorityClassModel(majority_class)

# # "Train" the model (no actual training needed for this dummy model)

# Make predictions on validation and test sets
y_val_pred = base_model.predict(X_val)  # Replace val_df with actual image data if available
y_test_pred = base_model.predict(X_test)  # Replace test_df with actual image data if available


# Evaluate the model, rounded to 3 decimals
val_accuracy = base_model.evaluate(X_val, y_val)
test_accuracy = base_model.evaluate(X_test, y_test)

print(f"Validation Accuracy: {val_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
df_model_accuracy = pd.DataFrame({
    'Model': ['Baseline'],
    'Training Accuracy': [base_model.evaluate(X_train, y_train)],
    'Validation Accuracy': [val_accuracy],
    'Test Accuracy': [test_accuracy],
    'Color?': ['Yes']
})
df_model_accuracy

In [ ]:
grey_row = {
    'Model': 'Baseline',
    'Training Accuracy': base_model.evaluate(X_train_grey, y_train_grey),
    'Validation Accuracy': val_accuracy,
    'Test Accuracy': test_accuracy,
    'Color?': 'No'
}

# Using append (deprecated)
# df = df.append(new_row, ignore_index=True)

# Using concat (recommended)
df_model_accuracy = pd.concat([df_model_accuracy, pd.DataFrame([grey_row])], ignore_index=True)
df_model_accuracy

# Build our Basic Binary Model With the Balanced Training Set


In [ ]:
# define an instance of the early_stopping class
early_stopping = tf.keras.callbacks.EarlyStopping(
monitor='accuracy',
verbose=1,
patience=4,
mode='max',
restore_best_weights=True)

In [ ]:
class BasicCNNModel:
    def __init__(self, input_shape, learning_rate=0.001):
        self.model = self.create_cnn_model()
        self.model.build(input_shape=input_shape)
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                           loss=tf.keras.losses.BinaryCrossentropy(),
                           metrics=['accuracy'])

    def create_cnn_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(filters=12, kernel_size=(4, 4), strides=(1, 1), padding='same',
                                         data_format='channels_last', name='conv_1', activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.3))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        return model

    def summary(self):
        self.model.summary()

    def fit(self, X_train, y_train, epochs, validation_data, callbacks):
        self.model.fit(X_train, y_train, epochs=epochs, validation_data=validation_data, callbacks=callbacks)

    def evaluate(self, X, y):
        return self.model.evaluate(X, y)

    def predict(self, X):
        return self.model.predict(X)

# Example usage
# Assuming X_train_balanced, y_train_balanced, X_val, y_val are already defined
input_shape = (None, 180, 120, 3)

cnn_model_color = BasicCNNModel(input_shape=input_shape, learning_rate = .0001)
cnn_model_color.summary()
cnn_model_color.fit(X_train_balanced, y_train_balanced, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
train_acc_cnn_color = cnn_model_color.evaluate(X_train_balanced, y_train_balanced)
val_acc_cnn_color = cnn_model_color.evaluate(X_val, y_val)

print("Training Accuracy:", train_acc_cnn_color[1])
print("Validation Accuracy:", val_acc_cnn_color[1])

#### Same Model on the Grey-Scale Data

In [ ]:
input_shape_grey = (None, 180, 120, 1)

cnn_model_grey = BasicCNNModel(input_shape=input_shape_grey, learning_rate = .0001)
cnn_model_grey.summary()
cnn_model_grey.fit(X_train_balanced_grey, y_train_balanced_grey, epochs=10, validation_data=(X_val_grey, y_val_grey), callbacks=[early_stopping])

# Evaluate the model
train_acc_cnn_grey = cnn_model_grey.evaluate(X_train_balanced_grey, y_train_balanced_grey)
val_acc_cnn_grey = cnn_model_grey.evaluate(X_val_grey, y_val_grey)

print("Training Accuracy Greyscale:", train_acc_cnn_grey[1])
print("Validation Accuracy Greyscale:", val_acc_cnn_grey[1])

In [ ]:
test_acc_cnn = cnn_model.evaluate(X_test, y_test)

new_row = {'Model': 'Basic CNN', 'Training Accuracy': train_acc_cnn[1], 'Validation Accuracy': val_acc_cnn[1],
    'Test Accuracy': test_acc_cnn[1]}

# Using append (deprecated)
# df = df.append(new_row, ignore_index=True)

# Using concat (recommended)
df_model_accuracy = pd.concat([df_model_accuracy, pd.DataFrame([new_row])], ignore_index=True)
df_model_accuracy

In [ ]:
# prompt: using certain indices of X_train, plot the image with the real label next to the predicted label the model would give it

# Choose 5 random indices from indices_to_remove
random_indices = np.random.choice(indices_to_remove, size=5, replace=False)

# Create a figure and axes
fig, axes = plt.subplots(1, 5, figsize=(20, 5))

# Iterate over the random indices and display the images with predictions
for i, idx in enumerate(random_indices):
    image = X_train[idx]
    # original_label = df_binary.loc[idx, 'original_label']
    true_label = y_train[idx]

    # Make a prediction for the image
    prediction = cnn_model.predict(np.expand_dims(image, axis=0))
    predicted_label = "Emotive" if prediction > 0.5 else "Neutral"

    # Display the image
    axes[i].imshow(image)
    axes[i].set_title(f"True: {true_label}\nPredicted: {predicted_label}")
    axes[i].axis('off')

plt.show()

## Create Complex CNN (Hima's Model)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

class ComplexCNNModel:
    def __init__(self, input_shape, learning_rate=0.001):
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.model = self.create_complex_cnn_model()

    def create_complex_cnn_model(self):
        model = Sequential()
        # 1st Convolutional layer
        model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=self.input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # 2nd Convolutional layer
        model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # 3rd Convolutional layer
        model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # Flattening layer
        model.add(Flatten())
        # 1st Dense layer
        model.add(Dense(units=512, activation='relu'))
        model.add(Dropout(0.5))
        # 2nd Dense layer
        model.add(Dense(units=256, activation='relu'))
        model.add(Dropout(0.5))
        # Output layer
        model.add(Dense(units=1, activation='sigmoid'))  # For binary classification
        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        return model

    def summary(self):
        self.model.summary()

    def fit(self, X_train, y_train, epochs, batch_size, validation_data, callbacks):
        self.history = self.model.fit(X_train, y_train,
                                      epochs=epochs,
                                      batch_size=batch_size,
                                      validation_data=validation_data,
                                      callbacks=callbacks)

    def evaluate(self, X, y):
        return self.model.evaluate(X, y)

    def predict(self, X):
        return self.model.predict(X)

# Example usage
# Assuming X_train, y_train, X_val, y_val are already defined and preprocessed
input_shape = (180, 120, 3)  # Update this to match your input data shape

cnn_model2 = ComplexCNNModel(input_shape=input_shape)
cnn_model2.summary()

cnn_model2.fit(X_train_balanced, y_train_balanced, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
train_loss_complex, train_accuracy_complex = cnn_model2.evaluate(X_train_balanced, y_train_balanced)
val_loss_complex, val_accuracy_complex = cnn_model2.evaluate(X_val, y_val)

print("Training loss:", train_loss_complex)
print("Training accuracy:", train_accuracy_complex)
print("Validation loss:", val_loss_complex)
print("Validation accuracy:", val_accuracy_complex)

# # Predict on new data
# y_pred = cnn_model.predict(X_val)
# print("Predictions:", y_pred)

In [ ]:
test_acc_cnn2 = cnn_model.evaluate(X_test, y_test)[1]

new_row2 = {'Model': 'Complex CNN', 'Training Accuracy': train_accuracy_complex, 'Validation Accuracy':val_accuracy_complex,
    'Test Accuracy': test_acc_cnn2}

# Using append (deprecated)
# df = df.append(new_row, ignore_index=True)

# Using concat (recommended)
df_model_accuracy = pd.concat([df_model_accuracy, pd.DataFrame([new_row2])], ignore_index=True)
df_model_accuracy